In [1]:
#importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Read data files
comments_attack = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_attack = pd.read_csv('attack_annotations.tsv',  sep = '\t')
comments_aggression = pd.read_csv('aggression_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_aggression = pd.read_csv('aggression_annotations.tsv',  sep = '\t')
comments_toxicity = pd.read_csv('toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_toxicity = pd.read_csv('toxicity_annotations.tsv',  sep = '\t')

In [3]:
# labels a comment if the majority of annoatators did so
labels_attack = annotations_attack.groupby('rev_id')['attack'].mean() > 0.5
labels_aggression = annotations_aggression.groupby('rev_id')['aggression'].mean() > 0.5
labels_toxicity = annotations_toxicity.groupby('rev_id')['toxicity'].mean() > 0.5


In [4]:
# join labels and comments
comments_attack['label'] = labels_attack
comments_aggression['label'] = labels_aggression
comments_toxicity['label'] = labels_toxicity

In [5]:
# Take only Attack, Aggression, Toxicity
comments_attack = comments_attack.query("label == True")
comments_aggression = comments_aggression.query("label == True")
comments_toxicity = comments_toxicity.query("label == True")

In [6]:
# labels: Attack = 0, Aggression =1, Toxicity = 2
comments_attack['label'] = comments_attack['label']*0
comments_aggression['label'] = comments_aggression['label']*1
comments_toxicity['label'] = comments_toxicity['label']*2


In [7]:
len(comments_attack)

13590

In [8]:
# Concatenation of the three data sets
dataframe = pd.concat([comments_attack, comments_aggression, comments_toxicity], axis = 0)


In [9]:
# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))

In [11]:
dataframe.iloc[1002]['comment']

'to pull their heads from their collective arses'

In [12]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [13]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  34987
test comments length:  8747


In [25]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(dataframe['comment'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [26]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [14]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [19]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid):
    

   
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)
        
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [20]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))

NB, Count Vectors:   accuracy: 0.3437750085743684      f1 score: 0.3118598812636129


In [21]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, Count Vectors:   accuracy: 0.3437750085743684      f1 score: 0.3118598812636129
NB, WordLevel TF-IDF:   accuracy: 0.2560878015319538     f1 score: 0.22216247424119906
NB, N-Gram Vectors:   accuracy: 0.2816965816851492     f1 score: 0.23907910356412748
NB, CharLevel Vectors:   accuracy: 0.2981593689264891   f1 score: 0.26497631292940804


In [22]:
# Linear Classifier on Count Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count, valid_y)
print("LR, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LR, Count Vectors:   accuracy: 0.33131359323196524   f1 score: 0.17108206844035737
LR, WordLevel TF-IDF:   accuracy: 0.22510575054304333   f1 score: 0.21383260886750863
LR, N-Gram Vectors:   accuracy: 0.26329027095004004   f1 score: 0.23339983237368095
LR, CharLevel Vectors:   accuracy: 0.24111123813879043   f1 score: 0.22624874099899567


In [ ]:
# SVM Classifier on Count Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count,valid_y)
print("SVM, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("SVM, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("SVM, N-Gram Vectors TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("SVM, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, Count Vectors:   accuracy: 0.13799016805761977   f1 score: 0.12199894437396669


C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVM, WordLevel TF-IDF:   accuracy: 0.3528066765748257   f1 score: 0.18402119562408722
SVM, N-Gram Vectors TF-IDF:   accuracy: 0.3528066765748257   f1 score: 0.18402119562408722


In [39]:
# Feed forward NN with 1 hidden layer
def model_FF(xtrain, ytrain, xvalid, yvalid, hidden_size, epochs =1):
    # create input layer 
    input_layer = layers.Input((xtrain.shape[1], ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(hidden_size, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(3, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    classifier.fit(xtrain, ytrain,
                  batch_size=256,
                  epochs=epochs,
                  shuffle = True)
    # scores of the classifier
    predictions = classifier.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = classifier.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [40]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [43]:

# NN Classifier on Count Vectors
accuracy, f1score = model_FF(xtrain_count, train_y_onehot, xvalid_count, valid_y_onehot, 100)
print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# NN Classifier on Word Level TF IDF Vectors
accuracy, f1score = model_FF(xtrain_tfidf, train_y_onehot, xvalid_tfidf, valid_y_onehot, 100)
print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# NN Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = model_FF(xtrain_tfidf_ngram, train_y_onehot, xvalid_tfidf_ngram, valid_y_onehot, 100)
print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# NN Classifier on Count Vectors
accuracy, f1score = model_FF(xtrain_tfidf_ngram_chars, train_y_onehot, xvalid_tfidf_ngram_chars, valid_y_onehot, 100)
print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/1
34987/34987 [==============================] - 2s 64us/step - loss: 1.6664 - acc: 0.3321
NN, Count Vectors accuracy:0.32193895051094196     f1 score: 0.32204448781604045
Epoch 1/1
34987/34987 [==============================] - 4s 127us/step - loss: 1.0998 - acc: 0.3299
NN, Count Vectors accuracy:0.32674059678625894     f1 score: 0.25234666100189335
Epoch 1/1
34987/34987 [==============================] - 4s 114us/step - loss: 1.1002 - acc: 0.3288
NN, Count Vectors accuracy:0.3318852177991777     f1 score: 0.21099053638215218
Epoch 1/1
34987/34987 [==============================] - 6s 185us/step - loss: 1.0988 - acc: 0.3448
NN, Count Vectors accuracy:0.3528066765918614     f1 score: 0.18402119562408722


In [48]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [49]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
34987/34987 [==============================] - 51s 1ms/step - loss: 1.0993 - acc: 0.3440
Epoch 2/3
34987/34987 [==============================] - 47s 1ms/step - loss: 1.0971 - acc: 0.3544
Epoch 3/3
34987/34987 [==============================] - 47s 1ms/step - loss: 1.0968 - acc: 0.3555
CNN, Word Embeddings acuuracy accuracy:0.3506345032752957     f1 score: 0.18730536879517634


In [51]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [52]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
34987/34987 [==============================] - 195s 6ms/step - loss: 1.1013 - acc: 0.3416
Epoch 2/3
34987/34987 [==============================] - 215s 6ms/step - loss: 1.0987 - acc: 0.3510
Epoch 3/3
34987/34987 [==============================] - 208s 6ms/step - loss: 1.0978 - acc: 0.3552
LSTM, Word Embeddings accuracy:0.3434320338503952     f1 score: 0.2693006603666671
